### Steps
- Use a list of **app_id** to get info from Steam crawler and insert **app_id**, **game_name**, **header_img_url**, **total_positive**, **total_negative**, **total_reviews** into table **games** and insert **game_id**, **review**, **recommended**, **time** into table **reviews**.
- Preprocess **review** in table **reviews** (add_missing_punct, replace_bullets, remove_url, remove_html_tags, normalize_single_quote, remove_non_ascii, remove_ansi_escape_sequences, remove_multi_whitespaces), then tokenize_sent and remove_leading_symbols to insert **review_id**, **sent** into table **sents**.
- Preprocess **sent** in table **sents** (lowercase, expand contractions, remove_digits, remove_symbols, remove_multi_whitespaces, lemmatize_text, remove_stopwords) to create **sent_prep** in table **sents**.
	- Use **sent_prep**, **review_id** in table **sents** to insert **review_prep** in table **reviews** by joining **sent_prep**.
- Use **review_prep** in table **reviews** to calculate special bigrams frequency, get 50 most frequent keywords.
- Insert **kw**, **freq** into table **kws**.
- Embed 50 keywords using S-BERT and cluster them using agglomerative clustering with a distance_threshold=0.6. 
	- Insert **cluster_name** (name of the most frequent keyword in cluster) into table **clusters**.
	- Insert **cluster_id** in table **kws**.
- Loop through **sent_prep** in table **sents**, fuzzy-match each **kw** in table **kws**. 
    - Insert **cluster_id**, **sent_id** in table **clusters_sents**t to link table **clusters** and **sents**.
- Remove all **sent_id** in table **sents** if they don't exist in table **clusters_sents**.
    - Insert **score_flair**, **score_vader**, **recommended**, **score_total**, **sent_embedding** in table **sents**.

### Step 1:

Use a list of **app_id** to get info from Steam crawler and insert **app_id**, **game_name**, **header_img_url**, **total_positive**, **total_negative**, **total_reviews** into table **games** and insert **game_id** (fk), **review**, **recommended**, **time** into table **reviews**.

In [1]:
import requests 
import pandas as pd
import numpy as np
import download_steam_reviews
import sqlite3

In [10]:
conn = sqlite3.connect('./data/steam_reviews.db') 
cursor = conn.cursor()

In [11]:
def print_cursor(cursor): 
    print(cursor.execute("""
        select * from games;
    """).fetchall())

In [12]:
print_cursor(cursor)

[(1, 428550, 'Momodora: Reverie Under the Moonlight', 'https://cdn.cloudflare.steamstatic.com/steam/apps/428550/header.jpg', 3561, 269, 3830), (2, 367520, 'Hollow Knight', 'https://cdn.cloudflare.steamstatic.com/steam/apps/367520/header.jpg', 4877, 123, 5000)]


In [19]:
print(cursor.execute("""
        select game_id from games 
        where app_id=367520;
    """).fetchall()[0][0])

2


In [5]:
# 428550 - Momodora: Reverie under the Moonlight
# 367520 - Hollow Knight
app_ids = [428550, 367520]

In [6]:
### get game_name ###

def get_app_list(): 
    app_list_url = 'https://api.steampowered.com/ISteamApps/GetAppList/v2/'
    resp_data = requests.get(app_list_url)
    return resp_data.json()

def get_name(app_id, app_list): 
    for app in app_list['applist']['apps']: 
        if app['appid'] == app_id: 
            return app['name']

In [7]:
### get header_img_url  

def get_header_img_url(app_id): 
    return f'https://cdn.cloudflare.steamstatic.com/steam/apps/{app_id}/header.jpg'

In [9]:
### get total_positive, total_negative, total_reviews in Crawler for table games 
### get game_id, review, recommended, time in Crawler for table games 

app_list = get_app_list()

request_params = {
    'language': 'english'
}

# load or download new 
load_mode = True

for app_id in app_ids: 
    game_tuples = [] 

    game_name = get_name(app_id, app_list)
    
    header_img_url = get_header_img_url(app_id)
    
#     with conn:
#         cursor.execute("""INSERT INTO games (app_id, game_name, header_img_url) VALUES (?, ?, ?);""", 
#                        (app_id, game_name, header_img_url))
    
    total_positive, total_negative, total_reviews = 0, 0, 0
    
    if load_mode: 
        review_dict = download_steam_reviews.load_review_dict(app_id)['reviews'].values()
    else: 
        review_dict = download_steam_reviews.download_reviews_for_app_id(app_id, 
                                                                     chosen_request_params=request_params, 
                                                                     reviews_limit=5000)[0]['reviews'].values()
    
    for review_dict_value in review_dict: 
        total_reviews += 1
    
        voted_up = 1 if review_dict_value['voted_up'] else 0
    
        if voted_up: 
            total_positive += 1
        else: 
            total_negative += 1
     
        review = review_dict_value['review']
        recommended = voted_up
        time = review_dict_value['timestamp_updated']
    
        with conn:
            cursor.execute("""INSERT INTO reviews (review, recommended, time, game_id) VALUES 
                                (?, ?, ?, (SELECT game_id from games WHERE app_id=?));""",
                           (review, recommended, time, app_id))
            

    with conn:
        cursor.execute("""UPDATE games 
                            SET total_positive=?,
                                total_negative=?,
                                total_reviews=?
                            WHERE app_id=?;""",
                       (total_positive, total_negative, total_reviews, app_id))

### Step 2:

Preprocess **review** in table **reviews** (add_missing_punct, replace_bullets, remove_url, remove_html_tags, normalize_single_quote, remove_non_ascii, remove_ansi_escape_sequences, remove_multi_whitespaces), then tokenize_sent and remove_leading_symbols to insert **review_id**, **sent** into table **sents**.
    
        

    

In [23]:
df_reviews = pd.read_sql_query("SELECT review_id, review from reviews;", conn)

In [24]:
df_reviews

,review_id,review
0,1,Metroidvania with some influences from Dark So...
1,2,It's ok. Frustrating mechanics turned me off o...
2,3,Cat
3,4,Has problems but overall pretty good. I'd sugg...
4,5,This is a very good and very short game. If yo...
...,...,...
8825,8826,it's ok
8826,8827,One of the most polished and engaging games I'...
8827,8828,"ahem\n""very hard owo"""
8828,8829,this game sic
